# Imports

In [ ]:
!pip install git+https://github.com/weaviate/weaviate-python-client@multi2multivec-weaviate

In [12]:
import fitz
from base64 import b64encode, b64decode
from concurrent.futures import ThreadPoolExecutor, as_completed
import csv
from datasets import load_dataset
import dspy
import io
import os
import pandas as pd
from pathlib import Path
import requests
import time
from urllib.parse import urlparse

# Get the FinanceBench Dataset from HuggingFace and B64 encode each page

### Download PDFs

In [ ]:
ds = load_dataset("PatronusAI/financebench")
df = ds["train"].to_pandas()
df.info()

In [ ]:
def download_single_pdf(args):
    """Download a single PDF from a URL"""
    url, output_folder, index, doc_name = args
    
    try:
        # Use doc_name for filename
        if doc_name:
            # Ensure it ends with .pdf
            if not doc_name.lower().endswith('.pdf'):
                filename = f"{doc_name}.pdf"
            else:
                filename = doc_name
        else:
            # Fallback if doc_name is missing
            filename = f"document_{index}.pdf"
        
        output_path = Path(output_folder) / filename
        
        # Download the file
        response = requests.get(url, timeout=30, stream=True)
        response.raise_for_status()
        
        # Save to file
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        return {
            "success": True,
            "url": url,
            "filename": filename,
            "path": str(output_path),
            "index": index,
            "doc_name": doc_name
        }
    
    except Exception as e:
        return {
            "success": False,
            "url": url,
            "error": str(e),
            "index": index,
            "doc_name": doc_name
        }


In [20]:
def download_pdfs_from_dataframe(df, url_column='doc_link', doc_name_column='doc_name',
                                  output_folder='financebench-pdfs', max_workers=4, 
                                  delay_between_requests=0.1):
    """Download PDFs from URLs in a DataFrame column"""
    
    # Create output folder
    output_path = Path(output_folder)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Get URLs
    if url_column not in df.columns:
        raise ValueError(f"Column '{url_column}' not found in DataFrame")
    
    if doc_name_column not in df.columns:
        raise ValueError(f"Column '{doc_name_column}' not found in DataFrame")
    
    urls = df[url_column].tolist()
    doc_names = df[doc_name_column].tolist()
    total_urls = len(urls)
    
    print(f"Starting download of {total_urls} PDFs to '{output_folder}'...")
    print(f"Using {max_workers} parallel workers\n")
    
    successful_downloads = []
    failed_downloads = []
    
    # Download in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(download_single_pdf, (url, output_folder, idx, doc_name)): idx 
            for idx, (url, doc_name) in enumerate(zip(urls, doc_names))
        }
        
        for future in as_completed(futures):
            result = future.result()
            
            if result['success']:
                successful_downloads.append(result)
                print(f"✓ [{result['index']+1}/{total_urls}] {result['filename']}")
            else:
                failed_downloads.append(result)
                print(f"✗ [{result['index']+1}/{total_urls}] {result['url']}")
                print(f"  Error: {result['error']}")
            
            time.sleep(delay_between_requests)
    
    # Summary
    print("\n" + "="*60)
    print(f"Download Summary:")
    print(f"  Total: {total_urls} | Successful: {len(successful_downloads)} | Failed: {len(failed_downloads)}")
    print("="*60)
    
    if failed_downloads:
        print("\nFailed downloads:")
        for failure in failed_downloads:
            print(f"  - {failure['url']}: {failure['error']}")
    
    return {
        "total": total_urls,
        "successful": successful_downloads,
        "failed": failed_downloads,
        "success_count": len(successful_downloads),
        "failure_count": len(failed_downloads)
    }

In [ ]:
results = download_pdfs_from_dataframe(
    df=df,                              # Your DataFrame
    url_column='doc_link',              # Column with URLs
    doc_name_column='doc_name',         # Column with document names
    output_folder='financebench-pdfs',  # Where to save PDFs
    max_workers=4,                      # Parallel downloads
    delay_between_requests=0.1          # Delay between requests (seconds)
)

print(f"\n✓ Downloaded {results['success_count']} PDFs successfully!")

### Base64 Encodings per Page

In [23]:
DPI = 300
FORMAT = "png"
PDF_FOLDER = "financebench-pdfs"
OUTPUT_CSV = "pdf_pages_base64.csv"
MAX_WORKERS = 4

In [25]:
def process_single_pdf(args):
    """Process a single PDF file and return base64 encoded images for each page"""
    pdf_file, dpi, fmt = args
    
    zoom = dpi / 72.0
    mat = fitz.Matrix(zoom, zoom)
    
    start = time.time()
    try:
        doc = fitz.open(pdf_file)
    except Exception as e:
        return {"error": str(e), "filename": pdf_file.name}
    
    filename = pdf_file.stem
    pages_data = []
    total_image_bytes = 0
    total_base64_bytes = 0
    
    for i, page in enumerate(doc, start=1):
        pix = page.get_pixmap(matrix=mat, alpha=False)
        buf = io.BytesIO()
        img_bytes = pix.pil_tobytes(format=fmt.upper())
        buf.write(img_bytes)
        byte_data = buf.getvalue()
        
        # Get size before base64 encoding
        image_size_bytes = len(byte_data)
        
        # Encode to base64
        base64_str = b64encode(byte_data).decode("utf-8")
        
        # Get size after base64 encoding
        base64_size_bytes = len(base64_str.encode('utf-8'))
        
        # Track totals
        total_image_bytes += image_size_bytes
        total_base64_bytes += base64_size_bytes
        
        pages_data.append({
            "source_pdf": pdf_file.name,
            "page_number": i,
            "base64": base64_str,
            "image_size_bytes": image_size_bytes,
            "base64_size_bytes": base64_size_bytes
        })
    
    doc.close()
    elapsed = time.time() - start
    
    # Calculate compression ratio
    compression_ratio = (total_base64_bytes / total_image_bytes) if total_image_bytes > 0 else 0
    
    print(f"✓ Processed {len(pages_data)} pages from {pdf_file.name} in {elapsed:.2f}s | "
          f"Image: {total_image_bytes/1024/1024:.2f}MB → Base64: {total_base64_bytes/1024/1024:.2f}MB "
          f"(ratio: {compression_ratio:.2f}x)")
    
    return {
        "filename": pdf_file.name,
        "pdf_stem": filename,
        "pages": pages_data,
        "num_pages": len(pages_data),
        "time": elapsed,
        "total_image_bytes": total_image_bytes,
        "total_base64_bytes": total_base64_bytes,
        "compression_ratio": compression_ratio
    }


In [26]:
def process_pdfs_to_base64(pdf_folder=PDF_FOLDER, dpi=DPI, fmt=FORMAT, 
                           max_workers=MAX_WORKERS, number_pdfs=None):
    """Process all PDFs in folder and convert each page to base64"""
    
    pdf_path = Path(pdf_folder)
    pdf_files = list(pdf_path.glob("*.pdf"))
    
    if not pdf_files:
        print(f"No PDF files found in {pdf_folder}")
        return []
    
    total_pdfs = len(pdf_files)
    
    if number_pdfs is not None:
        pdf_files = pdf_files[:number_pdfs]
        print(f"Processing {len(pdf_files)} of {total_pdfs} PDFs using {max_workers} workers...")
    else:
        print(f"Processing all {total_pdfs} PDFs using {max_workers} workers...")
    
    print(f"DPI: {dpi}, Format: {fmt}\n")
    
    failed_to_open = []
    all_results = []
    
    # Process PDFs in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_single_pdf, (pdf_file, dpi, fmt)): pdf_file 
            for pdf_file in pdf_files
        }
        
        for future in as_completed(futures):
            result = future.result()
            
            # Check if processing failed
            if "error" in result and "pages" not in result:
                failed_to_open.append(result["filename"])
                print(f"✗ Could not open {result['filename']}: {result['error']}")
            else:
                all_results.append(result)
    
    # Summary
    print("\n" + "="*60)
    print(f"Processing Summary:")
    print(f"  Total PDFs: {len(pdf_files)}")
    print(f"  Successfully processed: {len(all_results)}")
    print(f"  Failed: {len(failed_to_open)}")
    total_pages = sum(r['num_pages'] for r in all_results)
    total_img_bytes = sum(r['total_image_bytes'] for r in all_results)
    total_b64_bytes = sum(r['total_base64_bytes'] for r in all_results)
    print(f"  Total pages extracted: {total_pages}")
    print(f"  Total image size: {total_img_bytes/1024/1024:.2f} MB")
    print(f"  Total base64 size: {total_b64_bytes/1024/1024:.2f} MB")
    if total_img_bytes > 0:
        print(f"  Overall compression ratio: {total_b64_bytes/total_img_bytes:.2f}x")
    print("="*60)
    
    if failed_to_open:
        print("\nFailed to open:")
        for fname in failed_to_open:
            print(f"  - {fname}")
    
    return all_results


In [27]:
def save_results_to_csv(results, output_file=OUTPUT_CSV):
    """Save the base64 encodings to a CSV file"""
    
    if not results:
        print("No results to save")
        return
    
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['pdf_filename', 'pdf_stem', 'page_number', 'image_size_bytes', 
                      'base64_size_bytes', 'base64_encoding']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for pdf_result in results:
            for page in pdf_result['pages']:
                writer.writerow({
                    'pdf_filename': pdf_result['filename'],
                    'pdf_stem': pdf_result['pdf_stem'],
                    'page_number': page['page_number'],
                    'image_size_bytes': page['image_size_bytes'],
                    'base64_size_bytes': page['base64_size_bytes'],
                    'base64_encoding': page['base64']
                })
    
    print(f"\n✓ Saved base64 encodings to {output_file}")


In [42]:
results = process_pdfs_to_base64(
    pdf_folder=PDF_FOLDER,
    dpi=DPI,
    fmt=FORMAT,
    max_workers=MAX_WORKERS,
    number_pdfs=None  # Set to a number to limit processing, or None for all
)

Processing all 74 PDFs using 4 workers...
DPI: 300, Format: png

✗ Could not open KRAFTHEINZ_2019_10K.pdf: cannot open broken document
✓ Processed 181 pages from AMERICANWATERWORKS_2021_10K.pdf in 24.68s | Image: 77.74MB → Base64: 103.65MB (ratio: 1.33x)
✓ Processed 179 pages from JPMORGAN_2021Q1_10Q.pdf in 29.63s | Image: 104.53MB → Base64: 139.37MB (ratio: 1.33x)
✓ Processed 195 pages from AMCOR_2020_10K.pdf in 30.40s | Image: 85.88MB → Base64: 114.51MB (ratio: 1.33x)
✓ Processed 220 pages from MGMRESORTS_2022_10K.pdf in 33.80s | Image: 115.43MB → Base64: 153.91MB (ratio: 1.33x)
✓ Processed 92 pages from 3M_2023Q2_10Q.pdf in 13.40s | Image: 46.98MB → Base64: 62.63MB (ratio: 1.33x)
✓ Processed 73 pages from NETFLIX_2017_10K.pdf in 10.74s | Image: 35.57MB → Base64: 47.43MB (ratio: 1.33x)
✓ Processed 152 pages from AMERICANWATERWORKS_2020_10K.pdf in 24.51s | Image: 88.98MB → Base64: 118.64MB (ratio: 1.33x)
✓ Processed 213 pages from CVSHEALTH_2022_10K.pdf in 34.51s | Image: 125.04MB → B

In [43]:
save_results_to_csv(results, output_file=OUTPUT_CSV)


✓ Saved base64 encodings to pdf_pages_base64.csv


In [4]:
pdf_pages_df = pd.read_csv("pdf_pages_base64.csv")
pdf_pages_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10955 entries, 0 to 10954
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   pdf_filename       10955 non-null  object
 1   pdf_stem           10955 non-null  object
 2   page_number        10955 non-null  int64 
 3   image_size_bytes   10955 non-null  int64 
 4   base64_size_bytes  10955 non-null  int64 
 5   base64_encoding    10955 non-null  object
dtypes: int64(3), object(3)
memory usage: 513.6+ KB


In [11]:
print(pdf_pages_df["pdf_filename"].nunique())
print(len(pdf_pages_df) / pdf_pages_df["pdf_filename"].nunique())

72
152.15277777777777


In [12]:
print(pdf_pages_df["base64_size_bytes"].sum() / 1024 / 1024)

7488.567134857178


In [8]:
7488 / 10955

0.6835235052487448

# FinanceBench Recall per PDF test

In [5]:
# Group the records by pdf_filename
grouped_pdfs = pdf_pages_df.groupby("pdf_filename")
# Now `grouped` is a DataFrameGroupBy object, and you can iterate or aggregate as needed.
# For example, to see how many pages per PDF:
page_counts = grouped_pdfs.size()
print(page_counts.head(10))

pdf_filename
3M_2018_10K.pdf                       160
3M_2022_10K.pdf                       252
3M_2023Q2_10Q.pdf                      92
ACTIVISIONBLIZZARD_2019_10K.pdf       198
AES_2022_10K.pdf                      257
AMAZON_2017_10K.pdf                    85
AMAZON_2019_10K.pdf                    83
AMCOR_2020_10K.pdf                    195
AMCOR_2022_8K_dated-2022-07-01.pdf      9
AMCOR_2023Q2_10Q.pdf                   57
dtype: int64


In [6]:
questions = load_dataset("PatronusAI/financebench")["train"]

filtered_questions = []
for q in questions:
    # evidence_page_num is inside first evidence dict, if it exists
    evidence_page_num = None
    if q.get("evidence") and isinstance(q["evidence"], list) and len(q["evidence"]) > 0:
        evidence_page_num = q["evidence"][0].get("evidence_page_num")
    filtered_questions.append({
        "question": q.get("question"),
        "doc_name": q.get("doc_name"),
        "answer": q.get("answer"),
        "evidence_page_num": evidence_page_num
    })

filtered_questions[0]  # show an example of filtered row

{'question': 'What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.',
 'doc_name': '3M_2018_10K',
 'answer': '$1577.00',
 'evidence_page_num': 59}

In [ ]:
from pydantic import AnyHttpUrl
import weaviate
from weaviate.collections.classes.config import Configure, DataType, Property
from weaviate.config import AdditionalConfig, Timeout

weaviate_client = weaviate.connect_to_weaviate_cloud(
    weaviate_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=os.getenv("WEAVIATE_API_KEY"),
    additional_config=AdditionalConfig(timeout=Timeout(insert=1800)),
)

collection_name = "FinanceBenchPDF"

def drop_and_create_collection(collection_name):
    if weaviate_client.collections.exists(collection_name):
        weaviate_client.collections.delete(collection_name)

    collection = weaviate_client.collections.create(
        name=collection_name,
        properties=[
            Property(name="page_number", data_type=DataType.INT, skip_vectorization=True),
            Property(name="page_image", data_type=DataType.BLOB),
        ],
        vector_config=Configure.MultiVectors.multi2vec_weaviate(
            base_url=AnyHttpUrl("https://dev-embedding.labs.weaviate.io"),
            image_fields=["page_image"],
            model="ModernVBERT/colmodernvbert",
        ),
    )

    return collection

from pydantic import BaseModel

class PDFwithPageNum(BaseModel):
    page_number: int
    page_image: str

def upload_pdfs_to_weaviate(
    weaviate_collection,
    pdf_with_page_num
):
    with weaviate_collection.batch.fixed_size(20) as batch:
        for page in pdf_with_page_num:
            batch.add_object(
                properties={
                    "page_number": int(page.page_number),
                    "page_image": page.page_image
                },
            )

In [8]:
filtered_questions[0]

{'question': 'What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.',
 'doc_name': '3M_2018_10K',
 'answer': '$1577.00',
 'evidence_page_num': 59}

In [9]:
lm = dspy.LM(
    "openai/gpt-4.1-mini",
    cache=False,
    api_key=os.getenv("OPENAI_API_KEY")
)

dspy.configure(lm=lm)

class AssessAnswerAlignment(dspy.Signature):
    """Assess the alignment of a system answer with the ground truth answer."""

    question: str = dspy.InputField(description="The question to be answered.")
    ground_truth_answer: str = dspy.InputField(description="The correct answer to the question.")
    system_answer: str = dspy.InputField(description="The answer provided by the system.")
    alignment_score: bool = dspy.OutputField(description="True if the system answer is aligned with the ground truth answer, False otherwise.")

llm_as_judge = dspy.ChainOfThought(AssessAnswerAlignment)

question = "What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement."
ground_truth_answer = "$1577.00"
system_answer_1 = "The capital expenditure amount for 3M in FY2018 was $1,577 million."
system_answer_2 = "3M's capital expenditure for 2018 was $530 million."

print(llm_as_judge(question=question, ground_truth_answer=ground_truth_answer, system_answer=system_answer_1).alignment_score)
print(llm_as_judge(question=question, ground_truth_answer=ground_truth_answer, system_answer=system_answer_2).alignment_score)


True


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


False


In [10]:
from weaviate.agents.query import QueryAgent

qa = QueryAgent(
    client=weaviate_client,
    collections=["FinanceBenchPDF"],
    agents_host="https://dev-agents.labs.weaviate.io"
)

In [11]:
# Initialize metrics lists
recall_at_1_list = []
recall_at_5_list = []
recall_at_20_list = []
alignment_scores = []

upload_times = []

# Loop through each PDF and its group of pages
for pdf_filename, group in grouped_pdfs:
    print(f"Processing PDF filename: {pdf_filename}")
    print("Page numbers in this PDF:", len(list(group["page_number"])))

    # Convert each row in group to a PDFwithPageNum instance
    pdf_with_page_num = [
        PDFwithPageNum(
            page_number=row["page_number"],
            page_image=row["base64_encoding"]
        )
        for _, row in group.iterrows()
    ]

    # Drop and create (reset) the collection in Weaviate
    weaviate_collection = drop_and_create_collection(collection_name)

    import time
    start_time = time.time()
    # Upload all pages of this PDF to Weaviate
    upload_pdfs_to_weaviate(weaviate_collection, pdf_with_page_num)
    elapsed_time = time.time() - start_time
    upload_times.append(elapsed_time)
    print(f"Uploaded {len(pdf_with_page_num)} pages from {pdf_filename} to Weaviate in {elapsed_time:.2f} seconds.")

    lookup_filename = pdf_filename.split(".")[0]
    question_row = next((q for q in filtered_questions if q['doc_name'] == lookup_filename), None)

    if question_row is None:
        print(f"No question found for {lookup_filename}. Skipping.")
        continue

    query = question_row["question"]
    ground_truth_page_num = question_row["evidence_page_num"]

    print(f"Query: {query}")
    print(f"Looking for page {ground_truth_page_num}.")
    print("-"*20)

    results = weaviate_collection.query.near_text(
        query=query,
        return_metadata=["distance"],
        return_properties=["page_number"],
        limit=20
    )

    retrieved_page_numbers = [obj.properties["page_number"] for obj in results.objects]
    print("Retrieved page numbers:", retrieved_page_numbers)
    print("-"*20)

    # Calculate recall@1, @5, @20
    recall_at_1 = int(ground_truth_page_num in retrieved_page_numbers[:1])
    recall_at_5 = int(ground_truth_page_num in retrieved_page_numbers[:5])
    recall_at_20 = int(ground_truth_page_num in retrieved_page_numbers[:20])

    recall_at_1_list.append(recall_at_1)
    recall_at_5_list.append(recall_at_5)
    recall_at_20_list.append(recall_at_20)

    # Now get the answer from the agent
    response = qa.ask(query)
    print("QA Agent Response:\n", response.final_answer)

    ground_truth_answer = question_row["answer"]
    system_answer = response.final_answer

    alignment_score = llm_as_judge(
        question=query,
        ground_truth_answer=ground_truth_answer,
        system_answer=system_answer
    ).alignment_score

    alignment_scores.append(alignment_score)
    print(f"Alignment score: {alignment_score}")

    break

# Aggregate and print overall metrics
print("==== Aggregate Metrics ====")
print(f"Recall@1: {sum(recall_at_1_list)}/{len(recall_at_1_list)} = {sum(recall_at_1_list)/max(1,len(recall_at_1_list)):.3f}")
print(f"Recall@5: {sum(recall_at_5_list)}/{len(recall_at_5_list)} = {sum(recall_at_5_list)/max(1,len(recall_at_5_list)):.3f}")
print(f"Recall@20: {sum(recall_at_20_list)}/{len(recall_at_20_list)} = {sum(recall_at_20_list)/max(1,len(recall_at_20_list)):.3f}")

alignment_avg = sum(map(int, alignment_scores))/len(alignment_scores)
print(f"Avg. Alignment Score: {alignment_avg:.3f}")
print(f"Alignment Scores Breakdown: {alignment_scores}")

print(f"Average upload time: {sum(upload_times)/len(upload_times):.2f} seconds.")
print(f"Max upload time: {max(upload_times):.2f} seconds.\n\n")

Processing PDF filename: 3M_2018_10K.pdf
Page numbers in this PDF: 160
Uploaded 160 pages from 3M_2018_10K.pdf to Weaviate in 138.66 seconds.
Query: What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.
Looking for page 59.
--------------------
Retrieved page numbers: [59, 39, 26, 43, 42, 7, 15, 106, 49, 41, 58, 31, 44, 74, 61, 47, 126, 75, 14, 78]
--------------------
QA Agent Response:
 The FY2018 capital expenditure amount (purchases of property, plant, and equipment) for 3M is $1,577 million.
Alignment score: True
==== Aggregate Metrics ====
Recall@1: 1/1 = 1.000
Recall@5: 1/1 = 1.000
Recall@20: 1/1 = 1.000
Avg. Alignment Score: 1.000
Alignment Scores Breakdown: [True]
Average upload time: 138.66 seconds.
Max upload time: 138.66 seconds.




### The following results are from completing 35 PDFs with the script above

In [60]:
# Aggregate and print overall metrics
print("==== Aggregate Metrics ====")
print(f"Recall@1: {sum(recall_at_1_list)}/{len(recall_at_1_list)} = {sum(recall_at_1_list)/max(1,len(recall_at_1_list)):.3f}")
print(f"Recall@5: {sum(recall_at_5_list)}/{len(recall_at_5_list)} = {sum(recall_at_5_list)/max(1,len(recall_at_5_list)):.3f}")
print(f"Recall@20: {sum(recall_at_20_list)}/{len(recall_at_20_list)} = {sum(recall_at_20_list)/max(1,len(recall_at_20_list)):.3f}")

alignment_avg = sum(map(int, alignment_scores))/len(alignment_scores)
print(f"Avg. Alignment Score: {alignment_avg:.3f}")
print(f"Alignment Scores Breakdown: {alignment_scores}")

print(f"Average upload time: {sum(upload_times)/len(upload_times):.2f} seconds.")
print(f"Max upload time: {max(upload_times):.2f} seconds.\n\n")

==== Aggregate Metrics ====
Recall@1: 3/35 = 0.086
Recall@5: 6/35 = 0.171
Recall@20: 13/35 = 0.371
Avg. Alignment Score: 0.629
Alignment Scores Breakdown: [True, False, False, True, True, False, True, True, True, False, True, False, True, False, True, True, True, False, False, False, True, True, False, True, False, False, True, True, False, True, True, True, True, True, True]
Average upload time: 172.38 seconds.
Max upload time: 2038.15 seconds.


